# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
# TFIDF features

gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1] # learning rate can be removed since the default value for Gradient Boosting Classifier is 0.1
}

gs = GridSearchCV(gb, param, cv=5, n_jobs=-1) 
# n_jobs here doesn't mean that the model itself will be trained in parallel
# rather, multiple versions of the model (with different hyperparameter settings) will be trained in parallel

cv_fit = gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,547.057190,29.176858,0.440313,0.029984,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.966786,0.980251,0.971249,0.966757,0.966757,0.970360,0.005242,1
3,783.018954,8.602348,0.405287,0.020675,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965889,0.977558,0.971249,0.970350,0.965858,0.970181,0.004306,2
5,762.748471,5.593317,0.268796,0.063922,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.965889,0.976661,0.969452,0.967655,0.967655,0.969462,0.003771,3
0,400.643072,5.230354,0.453614,0.023817,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964991,0.976661,0.969452,0.963163,0.966757,0.968205,0.004710,4
2,496.267424,7.156155,0.409766,0.035983,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.963196,0.978456,0.968553,0.966757,0.964061,0.968205,0.005469,4


Note: The grid search using TFIDF features took about an hour.

In [4]:
# Count Vectorized features

gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150],
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1] # learning rate can be removed since the default value for Gradient Boosting Classifier is 0.1
}

gs = GridSearchCV(gb, param, cv=5, n_jobs=-1) 
# n_jobs here doesn't mean that the model itself will be trained in parallel
# rather, multiple versions of the model (with different hyperparameter settings) will be trained in parallel

cv_fit = gs.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,855.394277,9.280454,0.263505,0.050439,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.977558,0.967655,0.966757,0.972147,0.969463,0.004952,1
3,845.533526,8.496996,0.492794,0.109269,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.976661,0.968553,0.964960,0.971249,0.969283,0.004380,2
0,358.798395,7.588628,0.532420,0.102581,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.977558,0.971249,0.963163,0.967655,0.968744,0.005252,3
2,528.487484,7.401711,0.422428,0.029785,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.976661,0.967655,0.963163,0.970350,0.968564,0.004723,4
1,553.169916,3.702979,0.457813,0.074554,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.978456,0.969452,0.964061,0.966757,0.968564,0.005332,5


Note: The grid search using count vectorized features took about an hour.